In [84]:
import numpy as np
import mpmath as mp
from mpmath import workdps, hyper,mpf,mpc
from mpmath import gamma as mp_gamma, arg as mp_arg, fabs as mp_abs, exp as mp_exp, convert as convert_to_mp, sqrt as mp_sqrt

from src.phasr.physical_constants import constants
from src.phasr.utility.math import momentum
from src.phasr.nuclei.parameterizations.coulomb import rho_kappa
pi=np.pi

In [95]:
from src.phasr.nuclei.parameterizations.coulomb import eta_coulomb


def hyper1f1_scalar_arbitrary_precision(a,b,z,dps=15):
    with workdps(dps):
        return hyper([a],[b],z)
hyper1f1_vector_arbitrary_precision=np.vectorize(hyper1f1_scalar_arbitrary_precision,excluded=[3])
hyper1f1_vector_z_arbitrary_precision=np.vectorize(hyper1f1_scalar_arbitrary_precision,excluded=[0,1,3])

def hyper1f1_coulomb(r,kappa,Z,energy,mass,reg=+1,dps=15,alpha_el=constants.alpha_el):
    # r in fm
    r=r/constants.hc
    k=momentum(energy,mass)
    y=constants.alpha_el*Z*energy/k
    rho=rho_kappa(kappa,Z)
    #
    if np.isscalar(r):
        hyper1f1=hyper1f1_scalar_arbitrary_precision
    else:
        hyper1f1=hyper1f1_vector_z_arbitrary_precision
    #
    print("arg1=", reg*rho+1+1j*y)
    print("arg2=", 2*reg*rho+1)
    print("arg3=", 2j*k*r)
    return hyper1f1(reg*rho+1+1j*y,2*reg*rho+1,2j*k*r,dps=dps)


def mp_g_coulomb_scalar(r,kappa,Z,energy,mass,reg,pass_eta=None,pass_hyper1f1=None,dps_hyper1f1=15,alpha_el=constants.alpha_el):
    # r in fm

    if pass_eta is None:
        pass_eta = eta_coulomb(kappa,Z,energy,mass,reg,alpha_el=alpha_el)
    if pass_hyper1f1 is None:
        pass_hyper1f1=hyper1f1_coulomb(r,kappa,Z,energy,mass,reg,dps=dps_hyper1f1,alpha_el=alpha_el)
    
    energy=convert_to_mp(energy)
    mass=convert_to_mp(mass)
    pass_eta = convert_to_mp(pass_eta)
    pass_hyper1f1 = convert_to_mp(pass_hyper1f1)
    
    k=momentum(energy,mass)
    y=alpha_el*Z*energy/k
    rho=rho_kappa(kappa,Z)
    sigma=rho*reg
    
    prefactor=(-1)*np.sign(kappa)*mp_sqrt(2*(energy+mass)/k)
    
    r=r/constants.hc
    
    return float(prefactor*((2*k*r)**sigma)*(mp.exp(pi*y/2))*(mp.abs(mp_gamma(sigma+1j*y))/(mp_gamma(2*sigma+1)))* ((mp_exp(-1j*k*r+1j*pass_eta))*(sigma+1j*y)*pass_hyper1f1).real )

mp_g_coulomb =  np.vectorize(mp_g_coulomb_scalar,excluded=[1,2,3,4,5,6,7,8,9])

def g_coulomb(r,kappa,Z,energy,mass,reg,pass_eta=None,pass_hyper1f1=None,dps_hyper1f1=15,alpha_el=constants.alpha_el):
    # r in fm

    rho=rho_kappa(kappa,Z)
    sigma=rho*reg
    
    if 0<mp_gamma(2*sigma+1)<np.inf:
        k=momentum(energy,mass)
        y=alpha_el*Z*energy/k
        
        if pass_eta is None:
            pass_eta = eta_coulomb(kappa,Z,energy,mass,reg,alpha_el=alpha_el)
        if pass_hyper1f1 is None:
            pass_hyper1f1=hyper1f1_coulomb(r,kappa,Z,energy,mass,reg,dps=dps_hyper1f1,alpha_el=alpha_el)
        
        prefactor=(-1)*np.sign(kappa)*np.sqrt(2*(energy+mass)/k)
        
        r=r/constants.hc
        
        return float(prefactor*(mpf(2*k*r)**mpf(sigma))*(mp.exp(np.pi*y/2))*(mp_abs(mp_gamma(sigma+1j*y))/(mp_gamma(2*sigma+1)))*((mp.exp(-1j*k*r+1j*pass_eta))*(sigma+1j*y)*pass_hyper1f1).real)
    else:
        if np.isscalar(r):
            return mp_g_coulomb_scalar(r,kappa,Z,energy,mass,reg,pass_eta,pass_hyper1f1,dps_hyper1f1,alpha_el)
        else:
            return mp_g_coulomb(r,kappa,Z,energy,mass,reg,pass_eta,pass_hyper1f1,dps_hyper1f1,alpha_el)



In [108]:
g_coulomb(r=385.0,kappa=2200,Z=13,energy=1000,mass=0,reg=+1)

arg1= (2200.9999979546637+0.09486558340068942j)
arg2= 4400.9999959093275
arg3= 3902.1526526131347j


-5.291336516547298e-37